In [99]:
replaces = {'MES': 'Marzo', 'bloque': 'Bloque 1', 'piso': 'Piso 3', 'Tipo_Internacion': 'General adultos', 'DX_PRINCIPAL': 'N390', 'D_DX_PPAL': 'TRAUMATISMOS, ENVENENAMIENTO Y ALGUNAS OTRAS CONSECUENCIAS DE CAUSAS EXTERNAS', 'D_Capitulo_CIE10': 'S00-T98', 'RangoCapitulo': 'N39', 'Categoria': 'OTROS TRASTORNOS DEL SISTEMA URINARIO', 'D_subcapitulo_CIE10': 'Sin información', 'DescSubgrupoOncologia': 'Sin información', 'SubgrupoOncologia': 'INFECCION DE VIAS URINARIAS  SITIO NO ESPECIFICADO', 'ClasificacionCronico_DX_Principal': 'No crónica', 'DX_MEDICO': 'Z515', 'D_DX_MEDICO': 'ATENCION PALIATIVA', 'ClasificacionCronico_DX_Medico': 'No crónica', 'DX_R1': 'I10X', 'D_DX_R1': 'HIPERTENSION ESENCIAL (PRIMARIA)', 'ClasificacionCronico_DX_R1': 'No crónica', 'DX_R2': 'I10X', 'D_DX_R2': 'HIPERTENSION ESENCIAL (PRIMARIA)', 'ClasificacionCronico_DX_R2': 'No crónica', 'DX_R3': '-1', 'D_DX_R3': 'HIPERTENSION ESENCIAL (PRIMARIA)', 'ClasificacionCronico_DX_R3': 'No crónica', 'DX_F3': '-1', 'D_DX_F3': 'OTROS DOLORES ABDOMINALES Y LOS NO ESPECIFICADOS', 'ClasificacionCronico_DX_F3': 'No crónica', 'DX_F3_R1': '-1', 'DX_F3_R2': '-1', 'DX_F3_R3': '-1', 'DX_pre_cx': '-1', 'DX_pos_cx': '-1', 'DX_MUERTE': '-1', 'Subcapitulo_DX_Muerte': '-1', 'capitulo_DX_Muerte': '-1', 'Transfusiones': 'NO', 'Antioboticos': 'NO', 'UCI_UCE': 'NO'}
keys = ['DiasEstanciaServicio',
       'Antioboticos', 'Transfusiones', 'UCI_UCE', 'DX_pos_cx',
       'DX_pre_cx', 'DX_R3', 'EdadEgreso', 'NombreTipoDiagnostico']
test_keys = [i for i in replaces.keys() if i in keys]
test_values = [j for i,j in enumerate(replaces.values()) if list(replaces.keys())[i] in keys]
res = {}
for key in test_keys:
    for value in test_values:
        res[key] = value
        test_values.remove(value)
        break

print(res)

{'DX_R3': '-1', 'DX_pre_cx': '-1', 'DX_pos_cx': '-1', 'Transfusiones': 'NO', 'Antioboticos': 'NO', 'UCI_UCE': 'NO'}


In [131]:
import gradio as gr
import joblib
import pandas as pd
import numpy as np
from sklearn import preprocessing as ps

keys = ['DiasEstanciaServicio',
       'Antioboticos', 'Transfusiones', 'UCI_UCE', 'DX_pos_cx',
       'DX_pre_cx', 'DX_R3', 'EdadEgreso', 'NombreTipoDiagnostico']

load_randomForest=joblib.load("random_forest_model.pkl")
load_pca=joblib.load("pca_model.pkl")
antiencoder = joblib.load("Antioboticos_encoder.pkl")
try:
    antiencoder = joblib.load("Antioboticos_encoder.pkl")
except Exception as err:
    print(err)
trabsebcider = joblib.load("./encoders/Transfusiones_encoder.pkl")
uciuce = joblib.load("./encoders/UCI_UCE_encoder.pkl")
pre = joblib.load('./encoders/DX_pre_cx_encoder.pkl')
pos = joblib.load('./encoders/DX_pos_cx_encoder.pkl')
r3 = joblib.load('./encoders/DX_R3_encoder.pkl')

def calcular_prediccion(DiasEstanciaServicio,Antioboticos,Transfusiones,UCI_UCE,DX_pos_cx,DX_pre_cx,DX_R3, EdadEgreso, NombreTipoDiagnostico):
    data = [DiasEstanciaServicio,antiencoder.transform([Antioboticos]),trabsebcider.transform([Transfusiones]),
            uciuce.transform([UCI_UCE]),pre.transform([DX_pos_cx]),pre.transform([DX_pre_cx]),
            r3.transform([DX_R3]), EdadEgreso, NombreTipoDiagnostico]
    data = pd.DataFrame([data], columns = keys)
    
    pca_data=load_pca.transform(
        np.array(data
        )
    )
    return np.exp(load_randomForest.predict(pca_data))

DiasEstanciaServicio = gr.Number(label="DiasEstanciaServicio")
Antioboticos = gr.Dropdown(["SI", "NO"], label="Antioboticos")
Transfusiones = gr.Dropdown(["SI", "NO"], label="Transfusiones")
UCI_UCE = gr.Dropdown(["SI", "NO"], label="UCI_UCE")
DX_pos_cx = gr.Textbox(label="DX_pos_cx")
DX_pre_cx = gr.Textbox(label="DX_pre_cx")
DX_R3 = gr.Textbox(label="DX_R3")
EdadEngreso= gr.Number(label="EdadEngreso")
prediccion = gr.Textbox(label="Dias de estancia en el Hospital")
NombreTipoDiagnostico=gr.Dropdown([0, 1], label="NombreTipoDiagnostico")

demo = gr.Interface(
    fn=calcular_prediccion,
    inputs=[DiasEstanciaServicio,Antioboticos,Transfusiones,UCI_UCE,DX_pos_cx,DX_pre_cx,DX_R3,EdadEngreso,NombreTipoDiagnostico],
    outputs=[prediccion]
)

demo.launch()

c:\Users\ADMH2\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\base.py:348: InconsistentVersionWarning: Trying to unpickle estimator DecisionTreeRegressor from version 1.3.0 when using version 1.3.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
c:\Users\ADMH2\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\base.py:348: InconsistentVersionWarning: Trying to unpickle estimator RandomForestRegressor from version 1.3.0 when using version 1.3.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
c:\Users\ADMH2\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\base.py:348: InconsistentVersionWarning: Trying to un

Running on local URL:  http://127.0.0.1:7911

To create a public link, set `share=True` in `launch()`.


c:\Users\ADMH2\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\base.py:465: UserWarning: X does not have valid feature names, but PCA was fitted with feature names
  warnings.warn(
c:\Users\ADMH2\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\base.py:465: UserWarning: X does not have valid feature names, but PCA was fitted with feature names
  warnings.warn(
c:\Users\ADMH2\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\base.py:465: UserWarning: X does not have valid feature names, but PCA was fitted with feature names
  warnings.warn(
c:\Users\ADMH2\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\base.py:465: UserWarning: X does not have valid feature names, but PCA was fitted with feature names
  warnings.warn(


In [129]:
keys = ["DiasEstanciaServicio", "Antioboticos", "Transfusiones", "UCI_UCE", "DX_pos_cx", "DX_pre_cx", "DX_R3", "EdadEgreso", "NombreTipoDiagnostico"]

# Crear un DataFrame de Pandas a partir de una lista de listas
data = [1,1,1,1,1,1,1,1,1]
df = pd.DataFrame([data], columns=keys)
antiencoder.transform(["NO"])
antiencoder.transform(["NO"])
trabsebcider.transform(["NO"])
uciuce.transform(["SI"])
pre.transform(["-1"])
pre.transform(["-1"])
r3.transform(["R11X"])

array([1721])

In [78]:
pca_data=load_pca.transform(

np.array([1,1,1,1,1,1,1,1,1]).reshape(1,-1)

)
pca_data

np.exp(load_randomForest.predict(pca_data))

c:\Users\ADMH2\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\base.py:465: UserWarning: X does not have valid feature names, but PCA was fitted with feature names
  warnings.warn(


array([11.83036407])